# HTTP协议基础深入学习

## 📚 学习目标

通过本章学习，你将：
- 深入理解HTTP协议的工作原理
- 掌握HTTP请求和响应的完整结构
- 熟练使用各种HTTP方法和状态码
- 学会分析和构建HTTP消息
- 掌握HTTP头部的使用技巧
- 理解HTTP的无状态特性及其影响

## 🎯 本章重点
- HTTP协议基础概念
- 请求/响应消息结构
- HTTP方法语义和用法
- 状态码分类和含义
- URL结构和参数处理
- HTTP头部字段详解
- 实际应用和调试技巧


# 第一节：HTTP协议概述

## 1.1 什么是HTTP？

HTTP（HyperText Transfer Protocol）是**超文本传输协议**，是Web的基础通信协议。

### 核心特点：

#### 🌐 客户端-服务器模型
```
客户端 (浏览器/应用) ←→ 服务器
     发送请求           返回响应
```

#### 🔄 无状态协议
- 每个请求都是独立的
- 服务器不会记住之前的请求
- 需要通过Cookies、Session等机制维持状态

#### 📡 基于TCP/IP
- 运行在TCP协议之上
- 默认端口：80 (HTTP) / 443 (HTTPS)
- 可靠的数据传输

#### 📝 文本协议
- 人类可读的格式
- 易于调试和理解
- 基于ASCII字符集

## 1.2 HTTP的发展历史

| 版本 | 年份 | 主要特性 |
|------|------|----------|
| HTTP/0.9 | 1991 | 只支持GET方法，无头部信息 |
| HTTP/1.0 | 1996 | 添加POST、HEAD方法，支持头部 |
| HTTP/1.1 | 1997 | 持久连接，分块传输，更多方法 |
| HTTP/2 | 2015 | 二进制协议，多路复用，头部压缩 |
| HTTP/3 | 2022 | 基于QUIC，更快的连接建立 |

## 1.3 HTTP vs HTTPS

| 特性 | HTTP | HTTPS |
|------|------|-------|
| 安全性 | 明文传输 | SSL/TLS加密 |
| 端口 | 80 | 443 |
| 性能 | 更快 | 稍慢（加密开销） |
| SEO | 影响排名 | 搜索引擎偏好 |
| 证书 | 不需要 | 需要SSL证书 |


In [ ]:
# 第一节实践：分析HTTP通信过程
import requests
import urllib.parse
from typing import Dict, Any
import json

def demonstrate_http_basics():
    """演示HTTP协议的基本概念"""
    
    print("=" * 60)
    print("🌐 HTTP协议基础演示")
    print("=" * 60)
    
    # 1. URL结构分析
    print("\n1️⃣ URL结构分析")
    url = "https://api.github.com/users/octocat?tab=repositories&sort=updated#profile"
    parsed = urllib.parse.urlparse(url)
    
    print(f"完整URL: {url}")
    print(f"协议 (scheme): {parsed.scheme}")
    print(f"主机 (netloc): {parsed.netloc}")
    print(f"路径 (path): {parsed.path}")
    print(f"查询参数 (query): {parsed.query}")
    print(f"片段 (fragment): {parsed.fragment}")
    
    # 解析查询参数
    query_params = urllib.parse.parse_qs(parsed.query)
    print(f"解析后的查询参数: {dict(query_params)}")
    
    # 2. HTTP请求结构演示
    print("\n2️⃣ HTTP请求结构")
    print("典型的HTTP GET请求格式：")
    print("""
    GET /users/octocat HTTP/1.1          ← 请求行
    Host: api.github.com                 ← 必需头部
    User-Agent: Python-requests/2.28.1   ← 用户代理
    Accept: application/json             ← 接受的内容类型
    Authorization: Bearer token123       ← 认证信息
                                        ← 空行分隔
    (GET请求通常没有请求体)              ← 请求体
    """)
    
    # 3. HTTP响应结构演示
    print("\n3️⃣ HTTP响应结构")
    print("典型的HTTP响应格式：")
    print("""
    HTTP/1.1 200 OK                     ← 状态行
    Content-Type: application/json      ← 内容类型
    Content-Length: 1234                ← 内容长度
    Cache-Control: public, max-age=60   ← 缓存控制
    ETag: "abc123"                      ← 实体标签
                                       ← 空行分隔
    {"login": "octocat", ...}          ← 响应体
    """)

def analyze_real_http_request():
    """分析真实的HTTP请求和响应"""
    
    print("\n" + "=" * 60)
    print("🔍 真实HTTP请求分析")
    print("=" * 60)
    
    # 创建一个请求会话来观察HTTP通信
    session = requests.Session()
    
    # 设置自定义头部
    session.headers.update({
        'User-Agent': 'Python-Learning-HTTP/1.0',
        'Accept': 'application/json',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8'
    })
    
    try:
        # 发送请求
        print("\n📤 发送HTTP请求...")
        url = "https://httpbin.org/get"
        params = {
            'name': '张三',
            'age': 25,
            'city': '北京'
        }
        
        response = session.get(url, params=params)
        
        # 分析请求信息
        print(f"\n📋 请求信息:")
        print(f"请求URL: {response.url}")
        print(f"请求方法: {response.request.method}")
        print(f"请求头部:")
        for key, value in response.request.headers.items():
            print(f"  {key}: {value}")
        
        # 分析响应信息
        print(f"\n📥 响应信息:")
        print(f"状态码: {response.status_code}")
        print(f"状态文本: {response.reason}")
        print(f"响应头部:")
        important_headers = ['content-type', 'content-length', 'server', 'date']
        for header in important_headers:
            value = response.headers.get(header)
            if value:
                print(f"  {header}: {value}")
        
        # 响应内容分析
        print(f"\n📊 响应内容分析:")
        data = response.json()
        print(f"服务器接收到的参数: {data.get('args', {})}")
        print(f"服务器识别的来源IP: {data.get('origin', 'unknown')}")
        
        # 性能指标
        print(f"\n⚡ 性能指标:")
        print(f"响应时间: {response.elapsed.total_seconds():.3f} 秒")
        print(f"响应大小: {len(response.content)} 字节")
        
    except Exception as e:
        print(f"❌ 请求失败: {e}")

def demonstrate_http_methods():
    """演示不同HTTP方法的使用"""
    
    print("\n" + "=" * 60)
    print("🔧 HTTP方法演示")
    print("=" * 60)
    
    base_url = "https://httpbin.org"
    
    # GET方法
    print("\n1️⃣ GET方法 - 获取资源")
    try:
        response = requests.get(f"{base_url}/get", params={'message': 'Hello GET'})
        print(f"状态码: {response.status_code}")
        print(f"服务器收到的参数: {response.json().get('args', {})}")
    except Exception as e:
        print(f"GET请求失败: {e}")
    
    # POST方法
    print("\n2️⃣ POST方法 - 创建资源")
    try:
        data = {'name': '张三', 'email': 'zhangsan@example.com'}
        response = requests.post(f"{base_url}/post", json=data)
        print(f"状态码: {response.status_code}")
        print(f"服务器收到的JSON数据: {response.json().get('json', {})}")
    except Exception as e:
        print(f"POST请求失败: {e}")
    
    # PUT方法
    print("\n3️⃣ PUT方法 - 更新资源")
    try:
        data = {'id': 123, 'name': '李四', 'status': 'updated'}
        response = requests.put(f"{base_url}/put", json=data)
        print(f"状态码: {response.status_code}")
        print(f"服务器收到的JSON数据: {response.json().get('json', {})}")
    except Exception as e:
        print(f"PUT请求失败: {e}")
    
    # DELETE方法
    print("\n4️⃣ DELETE方法 - 删除资源")
    try:
        response = requests.delete(f"{base_url}/delete")
        print(f"状态码: {response.status_code}")
        print(f"响应URL: {response.json().get('url', '')}")
    except Exception as e:
        print(f"DELETE请求失败: {e}")

# 执行所有演示
demonstrate_http_basics()
analyze_real_http_request()
demonstrate_http_methods()


# 第二节：HTTP方法详解

## 2.1 HTTP方法概述

HTTP方法定义了对资源的不同操作类型。每个方法都有特定的语义和用途。

## 2.2 主要HTTP方法详解

### 🔵 GET - 获取资源

**用途**：从服务器获取资源
**特点**：
- 安全的（不会改变服务器状态）
- 幂等的（多次请求结果相同）
- 可被缓存
- 参数在URL中

**示例**：
```http
GET /api/users/123 HTTP/1.1
Host: example.com
Accept: application/json
```

### 🟢 POST - 创建资源

**用途**：向服务器提交数据，通常用于创建新资源
**特点**：
- 非安全的（会改变服务器状态）
- 非幂等的（多次请求可能产生不同结果）
- 不被缓存
- 数据在请求体中

**示例**：
```http
POST /api/users HTTP/1.1
Host: example.com
Content-Type: application/json

{
  "name": "张三",
  "email": "zhangsan@example.com"
}
```

### 🟡 PUT - 更新/创建资源

**用途**：更新或创建指定的资源
**特点**：
- 非安全的
- 幂等的（多次请求结果相同）
- 客户端指定资源位置

**示例**：
```http
PUT /api/users/123 HTTP/1.1
Host: example.com
Content-Type: application/json

{
  "id": 123,
  "name": "李四",
  "email": "lisi@example.com"
}
```

### 🔴 DELETE - 删除资源

**用途**：删除指定的资源
**特点**：
- 非安全的
- 幂等的
- 成功后通常返回204 No Content

**示例**：
```http
DELETE /api/users/123 HTTP/1.1
Host: example.com
```

### 🟠 PATCH - 部分更新

**用途**：对资源进行部分修改
**特点**：
- 非安全的
- 非幂等的
- 只更新指定字段

**示例**：
```http
PATCH /api/users/123 HTTP/1.1
Host: example.com
Content-Type: application/json

{
  "email": "newemail@example.com"
}
```

### 🔵 HEAD - 获取头部信息

**用途**：获取资源的头部信息（不返回响应体）
**特点**：
- 安全的
- 幂等的
- 用于检查资源是否存在或获取元数据

### 🔵 OPTIONS - 获取支持的方法

**用途**：获取服务器支持的HTTP方法
**特点**：
- 安全的
- 幂等的
- 常用于CORS预检请求

## 2.3 方法特性总结表

| 方法 | 安全性 | 幂等性 | 可缓存 | 请求体 | 常见用途 |
|------|--------|--------|--------|--------|----------|
| GET | ✅ | ✅ | ✅ | ❌ | 获取资源 |
| POST | ❌ | ❌ | ❌ | ✅ | 创建资源 |
| PUT | ❌ | ✅ | ❌ | ✅ | 更新/创建资源 |
| DELETE | ❌ | ✅ | ❌ | ❌ | 删除资源 |
| PATCH | ❌ | ❌ | ❌ | ✅ | 部分更新 |
| HEAD | ✅ | ✅ | ✅ | ❌ | 获取头部 |
| OPTIONS | ✅ | ✅ | ❌ | ❌ | 获取选项 |

## 2.4 重要概念解释

### 🛡️ 安全性（Safe）
方法不会改变服务器状态，可以安全地重复调用

### 🔄 幂等性（Idempotent）
多次执行相同操作的结果是一样的

### 💾 可缓存性（Cacheable）
响应可以被客户端或中间代理缓存


In [ ]:
# 第二节实践：深入理解HTTP方法

def demonstrate_http_method_characteristics():
    """演示HTTP方法的特性：安全性、幂等性、可缓存性"""
    
    print("=" * 60)
    print("🔬 HTTP方法特性深入分析")
    print("=" * 60)
    
    import requests
    import time
    
    base_url = "https://httpbin.org"
    
    # 1. 演示GET方法的幂等性和可缓存性
    print("\n1️⃣ GET方法特性演示")
    print("🔍 测试幂等性 - 多次GET请求结果应该相同")
    
    get_url = f"{base_url}/get"
    params = {'timestamp': '固定值', 'test': 'idempotent'}
    
    # 发送多次相同的GET请求
    for i in range(3):
        response = requests.get(get_url, params=params)
        print(f"第{i+1}次GET请求 - 状态码: {response.status_code}, URL: {response.json()['url']}")
    
    print("✅ 结论：GET请求是幂等的，多次请求结果相同")
    
    # 2. 演示POST方法的非幂等性
    print("\n2️⃣ POST方法特性演示")
    print("🔍 测试非幂等性 - 多次POST请求可能产生不同结果")
    
    post_url = f"{base_url}/post"
    
    for i in range(3):
        data = {
            'action': 'create_user',
            'timestamp': time.time(),  # 每次都不同
            'attempt': i + 1
        }
        response = requests.post(post_url, json=data)
        received_data = response.json().get('json', {})
        print(f"第{i+1}次POST请求 - 时间戳: {received_data.get('timestamp')}")
    
    print("✅ 结论：POST请求是非幂等的，每次请求可以产生不同结果")
    
    # 3. 演示PUT方法的幂等性
    print("\n3️⃣ PUT方法特性演示")
    print("🔍 测试幂等性 - 多次PUT相同数据应该产生相同结果")
    
    put_url = f"{base_url}/put"
    fixed_data = {
        'user_id': 123,
        'name': '张三',
        'email': 'zhangsan@example.com',
        'status': 'active'
    }
    
    for i in range(3):
        response = requests.put(put_url, json=fixed_data)
        received_data = response.json().get('json', {})
        print(f"第{i+1}次PUT请求 - 用户ID: {received_data.get('user_id')}, 状态: {received_data.get('status')}")
    
    print("✅ 结论：PUT请求是幂等的，多次请求同样的数据结果相同")

def compare_put_vs_post():
    """对比PUT和POST的区别"""
    
    print("\n" + "=" * 60)
    print("⚖️  PUT vs POST 详细对比")
    print("=" * 60)
    
    import requests
    
    base_url = "https://httpbin.org"
    
    print("\n📋 场景对比：创建用户资源")
    
    # POST创建资源 - 服务器决定资源标识
    print("\n🟢 POST方式 - 服务器生成资源ID")
    post_data = {
        'name': '张三',
        'email': 'zhangsan@example.com',
        'department': '技术部'
    }
    
    response = requests.post(f"{base_url}/post", json=post_data)
    print(f"POST请求发送的数据: {post_data}")
    print(f"服务器处理方式: 服务器会生成新的用户ID")
    print(f"适用场景: 用户注册、订单创建、文章发布")
    
    # PUT创建/更新资源 - 客户端指定资源标识
    print("\n🟡 PUT方式 - 客户端指定资源位置")
    put_data = {
        'user_id': 'emp_001',  # 客户端指定ID
        'name': '李四',
        'email': 'lisi@example.com',
        'department': '产品部'
    }
    
    response = requests.put(f"{base_url}/put", json=put_data)
    print(f"PUT请求发送的数据: {put_data}")
    print(f"服务器处理方式: 在指定位置创建或替换资源")
    print(f"适用场景: 配置更新、文件上传、用户资料管理")
    
    # 实际应用建议
    print("\n💡 选择建议:")
    print("📝 使用POST的场景:")
    print("  • 系统生成唯一标识符")
    print("  • 复杂的业务逻辑处理")
    print("  • 需要特殊验证或工作流")
    print("  • 示例：用户注册、下单、评论提交")
    
    print("\n📝 使用PUT的场景:")
    print("  • 客户端知道资源的确切位置")
    print("  • 需要幂等性保证")
    print("  • 完整替换资源内容")
    print("  • 示例：配置管理、文件上传、资料更新")

def demonstrate_http_headers():
    """演示重要的HTTP头部字段"""
    
    print("\n" + "=" * 60)
    print("📋 HTTP头部字段详解")
    print("=" * 60)
    
    import requests
    
    # 创建带有详细头部的请求
    headers = {
        'User-Agent': 'Python-Learning-Client/1.0 (Educational Purpose)',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Cache-Control': 'no-cache',
        'X-Custom-Header': 'Learning HTTP Headers',
        'Authorization': 'Bearer example-token-123'
    }
    
    try:
        response = requests.get('https://httpbin.org/headers', headers=headers)
        
        print("📤 发送的请求头部:")
        sent_headers = response.json()['headers']
        for key, value in sent_headers.items():
            print(f"  {key}: {value}")
        
        print(f"\n📥 收到的响应头部:")
        important_response_headers = [
            'content-type', 'content-length', 'server', 
            'date', 'cache-control', 'access-control-allow-origin'
        ]
        
        for header in important_response_headers:
            value = response.headers.get(header)
            if value:
                print(f"  {header}: {value}")
        
        # 头部字段解释
        print(f"\n📚 重要头部字段说明:")
        header_explanations = {
            'User-Agent': '标识客户端应用程序、操作系统和厂商',
            'Accept': '客户端可接受的内容类型',
            'Accept-Language': '客户端偏好的语言',
            'Authorization': '认证凭据',
            'Content-Type': '请求体的媒体类型',
            'Cache-Control': '缓存指令',
            'Content-Length': '请求体的字节长度'
        }
        
        for header, explanation in header_explanations.items():
            print(f"  • {header}: {explanation}")
            
    except Exception as e:
        print(f"❌ 头部演示失败: {e}")

# 执行所有演示
demonstrate_http_method_characteristics()
compare_put_vs_post()
demonstrate_http_headers()


# 第三节：HTTP状态码详解

## 3.1 状态码概述

HTTP状态码是服务器对请求处理结果的标准化表示。每个状态码都有特定的含义和使用场景。

## 3.2 状态码分类体系

### 📊 五大类别

| 类别 | 范围 | 含义 | 常见场景 |
|------|------|------|----------|
| 1xx | 100-199 | 信息性响应 | 协议升级、继续处理 |
| 2xx | 200-299 | 成功响应 | 请求成功处理 |
| 3xx | 300-399 | 重定向 | 需要进一步操作 |
| 4xx | 400-499 | 客户端错误 | 请求有问题 |
| 5xx | 500-599 | 服务器错误 | 服务器处理失败 |

## 3.3 详细状态码解析

### 🔵 1xx 信息性状态码

#### 100 Continue
- **含义**：服务器已收到请求头，客户端应继续发送请求体
- **使用场景**：大文件上传、分块传输
- **示例**：
```http
POST /upload HTTP/1.1
Content-Length: 1048576
Expect: 100-continue

HTTP/1.1 100 Continue
```

### ✅ 2xx 成功状态码

#### 200 OK
- **含义**：请求成功
- **使用场景**：GET、POST、PUT等成功且有返回内容
- **请求类型**：GET/POST/PUT/PATCH返回数据时

#### 201 Created
- **含义**：资源创建成功
- **使用场景**：POST创建新资源、PUT首次创建资源
- **特点**：通常返回新资源的位置（Location头）

#### 204 No Content
- **含义**：请求成功但无内容返回
- **使用场景**：DELETE成功、PUT/PATCH更新成功但不返回数据
- **特点**：响应体为空

### 🔄 3xx 重定向状态码

#### 301 Moved Permanently (永久重定向)
- **含义**：资源已永久移动到新位置
- **使用场景**：网站改版、域名变更
- **SEO影响**：搜索引擎会更新索引

#### 302 Found (临时重定向)
- **含义**：资源临时移动到新位置
- **使用场景**：临时跳转、登录后重定向
- **SEO影响**：保持原URL的搜索引擎权重

#### 304 Not Modified
- **含义**：资源未修改，使用缓存版本
- **使用场景**：条件请求、缓存验证
- **头部**：通常配合ETag或Last-Modified

### ❌ 4xx 客户端错误

#### 400 Bad Request
- **含义**：请求格式错误
- **常见原因**：JSON格式错误、参数缺失、数据类型错误
- **请求类型**：所有请求类型

#### 401 Unauthorized
- **含义**：需要认证
- **常见原因**：未提供认证信息、Token过期
- **请求类型**：所有需要认证的请求

#### 403 Forbidden
- **含义**：禁止访问
- **常见原因**：权限不足、IP被禁、资源受保护
- **与401区别**：已认证但无权限

#### 404 Not Found
- **含义**：资源不存在
- **常见场景**：页面不存在、资源ID不存在
- **请求类型**：GET/POST/PUT/DELETE等

#### 422 Unprocessable Entity
- **含义**：请求格式正确但语义错误
- **常见原因**：数据验证失败、业务逻辑错误
- **请求类型**：POST/PUT/PATCH请求

### 💥 5xx 服务器错误

#### 500 Internal Server Error
- **含义**：服务器内部错误
- **常见原因**：代码异常、数据库连接失败
- **请求类型**：所有请求类型

#### 502 Bad Gateway
- **含义**：网关错误
- **常见原因**：上游服务器无响应、负载均衡器问题
- **场景**：反向代理、微服务架构

#### 503 Service Unavailable
- **含义**：服务不可用
- **常见原因**：服务器维护、过载保护
- **特点**：通常是临时性的

## 3.4 状态码选择指南

### 📝 API设计最佳实践

#### GET请求状态码选择
```
成功获取资源 → 200 OK
资源不存在 → 404 Not Found
权限不足 → 403 Forbidden
未认证 → 401 Unauthorized
```

#### POST请求状态码选择
```
创建成功且返回数据 → 201 Created
创建成功无返回数据 → 204 No Content
数据验证失败 → 422 Unprocessable Entity
请求格式错误 → 400 Bad Request
```

#### PUT请求状态码选择
```
更新成功且返回数据 → 200 OK
更新成功无返回数据 → 204 No Content
首次创建成功 → 201 Created
资源不存在 → 404 Not Found
```

#### DELETE请求状态码选择
```
删除成功 → 204 No Content
资源已删除（幂等） → 204 No Content
资源不存在 → 404 Not Found
无删除权限 → 403 Forbidden
```


In [ ]:
# 第三节实践：HTTP状态码实战演练

def demonstrate_status_codes():
    """演示各种HTTP状态码的实际场景"""
    
    print("=" * 60)
    print("📊 HTTP状态码实战演练")
    print("=" * 60)
    
    import requests
    import json
    from requests.exceptions import RequestException
    
    base_url = "https://httpbin.org"
    
    # 1. 成功状态码演示
    print("\n✅ 2xx 成功状态码演示")
    
    # 200 OK
    try:
        response = requests.get(f"{base_url}/get")
        print(f"200 OK - GET请求成功: {response.status_code} {response.reason}")
        print(f"  响应内容长度: {len(response.content)} 字节")
    except Exception as e:
        print(f"GET请求失败: {e}")
    
    # 201 Created (模拟)
    try:
        data = {'name': '新用户', 'email': 'newuser@example.com'}
        response = requests.post(f"{base_url}/status/201", json=data)
        print(f"201 Created - 资源创建成功: {response.status_code} {response.reason}")
    except Exception as e:
        print(f"POST请求失败: {e}")
    
    # 204 No Content (模拟)
    try:
        response = requests.delete(f"{base_url}/status/204")
        print(f"204 No Content - 删除成功: {response.status_code} {response.reason}")
        print(f"  响应体是否为空: {len(response.content) == 0}")
    except Exception as e:
        print(f"DELETE请求失败: {e}")
    
    # 2. 重定向状态码演示
    print("\n🔄 3xx 重定向状态码演示")
    
    # 301 永久重定向
    try:
        response = requests.get(f"{base_url}/redirect-to?url={base_url}/get&status_code=301", 
                              allow_redirects=False)
        print(f"301 Moved Permanently: {response.status_code} {response.reason}")
        print(f"  重定向位置: {response.headers.get('Location', '未指定')}")
    except Exception as e:
        print(f"重定向测试失败: {e}")
    
    # 302 临时重定向
    try:
        response = requests.get(f"{base_url}/redirect-to?url={base_url}/get&status_code=302", 
                              allow_redirects=False)
        print(f"302 Found: {response.status_code} {response.reason}")
        print(f"  重定向位置: {response.headers.get('Location', '未指定')}")
    except Exception as e:
        print(f"重定向测试失败: {e}")
    
    # 3. 客户端错误演示
    print("\n❌ 4xx 客户端错误演示")
    
    # 400 Bad Request
    try:
        response = requests.get(f"{base_url}/status/400")
        print(f"400 Bad Request: {response.status_code} {response.reason}")
        print(f"  模拟场景: 请求格式错误、参数缺失")
    except Exception as e:
        print(f"400错误测试失败: {e}")
    
    # 401 Unauthorized
    try:
        response = requests.get(f"{base_url}/status/401")
        print(f"401 Unauthorized: {response.status_code} {response.reason}")
        print(f"  模拟场景: 未提供认证信息")
    except Exception as e:
        print(f"401错误测试失败: {e}")
    
    # 403 Forbidden
    try:
        response = requests.get(f"{base_url}/status/403")
        print(f"403 Forbidden: {response.status_code} {response.reason}")
        print(f"  模拟场景: 已认证但权限不足")
    except Exception as e:
        print(f"403错误测试失败: {e}")
    
    # 404 Not Found
    try:
        response = requests.get(f"{base_url}/status/404")
        print(f"404 Not Found: {response.status_code} {response.reason}")
        print(f"  模拟场景: 请求的资源不存在")
    except Exception as e:
        print(f"404错误测试失败: {e}")
    
    # 4. 服务器错误演示
    print("\n💥 5xx 服务器错误演示")
    
    # 500 Internal Server Error
    try:
        response = requests.get(f"{base_url}/status/500")
        print(f"500 Internal Server Error: {response.status_code} {response.reason}")
        print(f"  模拟场景: 服务器内部错误")
    except Exception as e:
        print(f"500错误测试失败: {e}")
    
    # 502 Bad Gateway
    try:
        response = requests.get(f"{base_url}/status/502")
        print(f"502 Bad Gateway: {response.status_code} {response.reason}")
        print(f"  模拟场景: 网关或代理服务器错误")
    except Exception as e:
        print(f"502错误测试失败: {e}")
    
    # 503 Service Unavailable
    try:
        response = requests.get(f"{base_url}/status/503")
        print(f"503 Service Unavailable: {response.status_code} {response.reason}")
        print(f"  模拟场景: 服务暂时不可用")
    except Exception as e:
        print(f"503错误测试失败: {e}")

def analyze_status_code_patterns():
    """分析不同API操作的状态码模式"""
    
    print("\n" + "=" * 60)
    print("🎯 API操作状态码模式分析")
    print("=" * 60)
    
    # CRUD操作的典型状态码模式
    patterns = {
        "用户管理API": {
            "GET /users": {
                "成功": "200 OK - 返回用户列表",
                "未授权": "401 Unauthorized - 需要登录",
                "禁止": "403 Forbidden - 无查看权限"
            },
            "GET /users/123": {
                "成功": "200 OK - 返回用户详情",
                "不存在": "404 Not Found - 用户不存在",
                "禁止": "403 Forbidden - 无查看该用户权限"
            },
            "POST /users": {
                "创建成功": "201 Created - 用户创建成功",
                "数据错误": "422 Unprocessable Entity - 邮箱已存在",
                "格式错误": "400 Bad Request - JSON格式错误"
            },
            "PUT /users/123": {
                "更新成功": "200 OK - 更新成功并返回数据",
                "更新成功(无返回)": "204 No Content - 更新成功无返回",
                "首次创建": "201 Created - 在指定位置创建用户",
                "不存在": "404 Not Found - 目标用户不存在"
            },
            "DELETE /users/123": {
                "删除成功": "204 No Content - 删除成功",
                "不存在": "404 Not Found - 用户不存在",
                "禁止删除": "403 Forbidden - 无删除权限"
            }
        }
    }
    
    for api_name, operations in patterns.items():
        print(f"\n📋 {api_name} 状态码模式:")
        for operation, scenarios in operations.items():
            print(f"\n  🔧 {operation}:")
            for scenario, description in scenarios.items():
                print(f"    • {scenario}: {description}")

def demonstrate_error_handling():
    """演示错误处理的最佳实践"""
    
    print("\n" + "=" * 60)
    print("🛠️ HTTP错误处理最佳实践")
    print("=" * 60)
    
    import requests
    from requests.exceptions import RequestException, ConnectionError, Timeout, HTTPError
    
    def robust_api_call(url, method='GET', **kwargs):
        """健壮的API调用示例"""
        try:
            response = requests.request(method, url, timeout=10, **kwargs)
            
            # 根据状态码进行不同处理
            if response.status_code == 200:
                print(f"✅ 请求成功: {response.status_code}")
                return {"success": True, "data": response.json()}
            
            elif response.status_code == 201:
                print(f"✅ 资源创建成功: {response.status_code}")
                return {"success": True, "data": response.json()}
            
            elif response.status_code == 204:
                print(f"✅ 操作成功(无内容返回): {response.status_code}")
                return {"success": True, "data": None}
            
            elif response.status_code == 400:
                print(f"❌ 请求格式错误: {response.status_code}")
                return {"success": False, "error": "请求参数有误"}
            
            elif response.status_code == 401:
                print(f"🔐 需要认证: {response.status_code}")
                return {"success": False, "error": "请先登录"}
            
            elif response.status_code == 403:
                print(f"🚫 权限不足: {response.status_code}")
                return {"success": False, "error": "权限不足"}
            
            elif response.status_code == 404:
                print(f"🔍 资源不存在: {response.status_code}")
                return {"success": False, "error": "请求的资源不存在"}
            
            elif response.status_code == 422:
                print(f"📝 数据验证失败: {response.status_code}")
                return {"success": False, "error": "数据验证失败"}
            
            elif 500 <= response.status_code < 600:
                print(f"💥 服务器错误: {response.status_code}")
                return {"success": False, "error": "服务器错误，请稍后重试"}
            
            else:
                print(f"❓ 未知状态码: {response.status_code}")
                return {"success": False, "error": f"未知错误: {response.status_code}"}
                
        except ConnectionError:
            print("🌐 网络连接错误")
            return {"success": False, "error": "网络连接失败"}
        
        except Timeout:
            print("⏰ 请求超时")
            return {"success": False, "error": "请求超时"}
        
        except Exception as e:
            print(f"❗ 未知错误: {e}")
            return {"success": False, "error": f"未知错误: {str(e)}"}
    
    # 测试不同状态码的处理
    test_cases = [
        ("https://httpbin.org/status/200", "GET"),
        ("https://httpbin.org/status/400", "GET"),
        ("https://httpbin.org/status/401", "GET"),
        ("https://httpbin.org/status/404", "GET"),
        ("https://httpbin.org/status/500", "GET"),
    ]
    
    print("\n🧪 错误处理测试:")
    for url, method in test_cases:
        status_code = url.split('/')[-1]
        print(f"\n测试状态码 {status_code}:")
        result = robust_api_call(url, method)
        print(f"处理结果: {result}")

# 执行所有演示
demonstrate_status_codes()
analyze_status_code_patterns()
demonstrate_error_handling()


# 第四节：HTTP头部字段详解

## 4.1 HTTP头部概述

HTTP头部字段提供了关于请求或响应的元信息，是HTTP协议的重要组成部分。

## 4.2 请求头部字段

### 🔑 认证相关
- **Authorization**: 认证凭据
- **Cookie**: 客户端存储的Cookie

### 📝 内容协商
- **Accept**: 客户端可接受的内容类型
- **Accept-Language**: 偏好的语言
- **Accept-Encoding**: 支持的编码方式
- **User-Agent**: 客户端标识

### 🔗 连接控制
- **Connection**: 连接管理选项
- **Keep-Alive**: 持久连接参数
- **Host**: 目标主机（HTTP/1.1必需）

## 4.3 响应头部字段

### 📊 内容信息
- **Content-Type**: 响应体的媒体类型
- **Content-Length**: 响应体字节长度
- **Content-Encoding**: 内容编码方式

### 💾 缓存控制
- **Cache-Control**: 缓存指令
- **ETag**: 实体标签
- **Last-Modified**: 最后修改时间
- **Expires**: 过期时间

### 🔒 安全相关
- **Set-Cookie**: 设置Cookie
- **Access-Control-Allow-Origin**: CORS跨域控制

## 4.4 条件请求头部

条件请求允许客户端只在满足特定条件时才处理请求，主要用于缓存和并发控制。

### 📅 基于时间的条件
- **If-Modified-Since**: 如果在指定时间后修改过
- **If-Unmodified-Since**: 如果在指定时间后未修改

### 🏷️ 基于ETag的条件
- **If-Match**: 如果ETag匹配
- **If-None-Match**: 如果ETag不匹配

## 4.5 自定义头部

### ✨ 常见自定义头部
- **X-Requested-With**: 标识AJAX请求
- **X-API-Key**: API密钥
- **X-Rate-Limit-***: 速率限制信息

## 📚 学习总结

通过本章学习，你已经掌握了：
- ✅ HTTP协议的基础概念和特点
- ✅ HTTP请求和响应的完整结构
- ✅ 各种HTTP方法的使用场景和特性
- ✅ HTTP状态码的分类和含义
- ✅ HTTP头部字段的作用和应用
- ✅ 实际的HTTP通信调试技巧

## 🎯 下一步

在掌握了HTTP协议基础后，建议继续学习：
1. **RESTful API设计理念** - 理解现代API设计规范
2. **JSON数据处理** - 掌握API数据交换格式
3. **API认证授权** - 学习安全的API访问控制
4. **API测试工具** - 使用Postman等工具测试API

继续加油！🚀
